In [ ]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("Financial Data Cleaning") \
    .getOrCreate()

# Load data
file_path = "../../../Data/GOOGL.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

In [ ]:
df.show(5)

In [ ]:
df.toPandas()

# Drop NA

In [ ]:
df_cleaned = df.dropna()
df_cleaned.toPandas() #.head(5)

# Fill NA

In [ ]:
from pyspark.sql.functions import col, mean

mean_values = df.select(mean(col('Open')).alias('mean_open'),
                        mean(col('High')).alias('mean_high'),
                        mean(col('Low')).alias('mean_low'),
                        mean(col('Close')).alias('mean_close'),
                        mean(col('Volume')).alias('mean_volume'),
                        mean(col('Adj Close')).alias('mean_adj_close')).collect()[0]   # first element in the list
print(mean_values)

df_filled = df.na.fill({'Open': mean_values['mean_open'],
                        'High': mean_values['mean_high'],
                        'Low': mean_values['mean_low'],
                        'Close': mean_values['mean_close'],
                        'Volume': mean_values['mean_volume'],
                        'Adj Close': mean_values['mean_adj_close']})

df_filled.toPandas().head(5)

# Create a new column

In [ ]:
from pyspark.sql.functions import expr

df_with_return = df_filled.withColumn("Daily_Return", (col("Close") - col("Open")) / col("Open"))
df_with_return.toPandas().head(5)

# Calculating the 7-day Moving Average (MA)

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import avg

# Specifies a window frame that includes the current row and the 6 preceding rows (7 rows total)
window_spec = Window.orderBy("Date").rowsBetween(-6, 0)

# Adds a new column "MA_7" to the DataFrame df_with_return that contains the 7-day moving average of the "Close" column.
df_with_ma = df_with_return.withColumn("MA_7", avg(col("Close")).over(window_spec))
df_with_ma.toPandas().head(32)


The warning you are seeing indicates that your window operation does not have a partition defined, causing all data to be moved to a single partition, which can degrade performance.

In [ ]:
df_with_ma.toPandas()[['Date','MA_7']].plot()

# Filtering the Data

In [ ]:
from pyspark.sql.functions import to_date, lit  # lit is Literal

df_filtered = df_with_ma.filter((col("Date") >= to_date(lit('2023-01-01'))) & 
                                (col("Date") <= to_date(lit('2023-12-31'))))
df_filtered.toPandas()

# Partitioning Example

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import avg, col

# Initialize Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Example DataFrame with a Region column
data = [("North", "2021-01-01", 10), ("North", "2021-01-02", 20), ("North", "2021-01-03", 30),
        ("South", "2021-01-01", 15), ("South", "2021-01-02", 25), ("South", "2021-01-03", 35),
        ("North", "2021-01-04", 40), ("North", "2021-01-05", 50), ("North", "2021-01-06", 60),
        ("South", "2021-01-04", 45), ("South", "2021-01-05", 55), ("South", "2021-01-06", 65)]
columns = ["Region", "Date", "Close"]

df_with_return = spark.createDataFrame(data, columns)
display(df_with_return.toPandas().sort_values('Region'))

# Ensure the DataFrame is sorted by Date within each Region
df_with_return = df_with_return.orderBy("Region", "Date")

# Define window specification with partitioning by Region and ordering by Date
window_spec = Window.partitionBy("Region").orderBy("Date").rowsBetween(-2, 0)

# Apply the moving average calculation
df_with_ma = df_with_return.withColumn("MA_3", avg(col("Close")).over(window_spec))

# Convert to Pandas
df_with_ma.toPandas()
